### 실습 개요
- 특정 유사도 수치를 기준으로 Video array를 분할하고 그룹화(Grouping)합니다.
- 그룹별 평균 유사도를 활용하여 해당 그룹의 활동성을 분석합니다.
- 분석 내용을 csv 파일에 저장합니다.

### 사전준비
- functions.py 안에 mp4 파일 decoding 함수들을 import 한다

In [ ]:
# functions.py 에 미리 정의된 함수들을 import 한다
from functions import video_2_ndarray

# numpy를 import 한다
import numpy as np

- mp4 file로부터 video array, 재생시간, frame개수 정보를 추출하기

In [ ]:
"""
함수 video_2_ndarray 를 사용하여 ../media/SampleVideo_640x360_5mb.mp4 의 video data를 ndarray 에 저장한다.
video array, 재생시간, frame 개수를 저장
"""
video_array, tot_duration, tot_frames = video_2_ndarray('../media/SampleVideo_640x360_5mb.mp4')

- 연속되는 두 frame들간의 유사도를 구해서 list 변수(similarity_list)에 저장

In [ ]:
from numpy import dot
from numpy.linalg import norm

# cosine similatiry 함수
def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

In [ ]:
prev_vector = None

similarity_list = []

for frame in video_array:

    # 1차원으로 reshape
    # 255로 나눔
    current_vector = (frame.reshape(-1) / 255)
    
    if prev_vector is not None:
        similarity = cos_sim(prev_vector, current_vector)
        similarity_list.append(similarity)
            
    prev_vector = current_vector.copy()

print('vector size : ',current_vector.shape)
print('similarity_list : ',similarity_list)

- pandas를 import 한다

In [ ]:
import pandas as pd

### 따라 해보기. pandas를 사용하여 영상 유사도 list를 grouping 하기
- 유사도가 낮아지는 지점을 기준으로 grouping 하기

#### 1. 유사도 DataFrame 생성

In [ ]:
import pandas as pd

# DataFrame 생성
df = pd.DataFrame(similarity_list, columns=['similarity'])

print(df)

In [ ]:
df.plot()

#### 2. 특정 유사도 값을 기준으로 '낮은 유사도'와 '높은 유사도' 두그룹으로 분할하기
-  ![Alt text for broken image link](../resources/pandas_plot.jpg)

In [ ]:
# 유사도 임계치 설정
lower_sim_threshold = 0.9

#### 3. 유사도 DataFrame에 2개 칼럼 추가
- lower_sim : 유사도 임계치 이하 여부

In [ ]:
# 유사도가 임계치 이하면 True(1) 아니면 False(0)
df['lower_sim'] = (df['similarity'] < lower_sim_threshold)

# 출력 시 '...' 없이 전체 출력
pd.set_option('display.max_rows', None)

# df 출력
print(df)

- cumsum : lower_sim의 누적 합계

In [ ]:
#lower_sim의 누적합(Cumulative Sum)을 구함
df['cumsum'] = df['lower_sim'].cumsum()

# df 출력
print(df)

# 출력 시 30 rows 출력 후 '...' 처리
pd.set_option('display.max_rows', 30)

In [ ]:
df.plot()

#### 4. cumsum값 을 기준으로 grouping
- 결과적으로 유사도가 낮은 순간(lower_sim_threshold 보다 작은 순간) group이 분할됨

In [ ]:
df_takes = df.groupby(df['cumsum'])

for group_number, group_df in df_takes:
    print('Group Name : ', group_number)
    print('Row Count : ', group_df.shape[0])
    print()

#### 5. Group별 시작 frame 번호, 종료 frame 번호, group 분할 시점의 유사도 조회

In [ ]:
for group_number, group_df in df_takes:
    min_frame_number = group_df.index.min()+1
    max_frame_number = group_df.index.max()+1
    min_similarity = group_df['similarity'].min()
    print(group_number, min_frame_number, max_frame_number, min_similarity)

#### 6. Group별 유사도 평균 계산
- 유사도 평균은 해당 Group의 영상이 얼마나 움직임이 많은지를 가늠함
- 좁은 범위에 조밀하게 분포한 값을 넓게 분포하도록 거듭제곱 변환 (Power Transformation)사용
- 1−x 변환을 이용해서 데이터의 범위를 [0, 1]로 유지하면서 값의 관점을 반대로 뒤집음

In [ ]:
for group_number, group_df in df_takes:

    min_similarity = group_df['similarity'].min()

    # 영상의 Take가 변경되는 지점의 similarity는 빼고 해당 Take(group DataFrame)의 similarity 평균을 구함
    similarity_mean = group_df[group_df['similarity'] > min_similarity]['similarity'].mean()

    # 1. 거듭제곱 변환 (Power Transformation)
    # 좁은 범위에 조밀하게 분포한 값을 넓게 분포하도록 변환(100 거듭제곱)
    # - data들이 0~1 사이 값이므로 거듭제곱 할수록 1에서 멀어지고 0 에 가까와짐. 
    # - data들이 1 근처에 조밀하게 분포하는 경우 효과적으로 분산시킴
    # 2. 1−x 변환
    # 데이터의 범위를 [0, 1]로 유지하면서 값의 관점을 반대로 뒤집음
    # - 값이 클수록 동적인 영상, 작을수록 정적인 영상이됨
    similarity_mean_power = similarity_mean**100
    activity_intensity = 1 - similarity_mean_power
    
    print(similarity_mean, similarity_mean_power, activity_intensity)

- 시각화하여 차이를 확인하기

In [ ]:
# 그룹별 유사도 평균 
mean = [ group_df[group_df['similarity'] > min_similarity]['similarity'].mean() \
        for group_number, group_df in df_takes ]

# 거듭제곱 변환 (100 거듭제곱)
mean_power = [ v**100 for v in mean ]

# 1-x 변환
activity = [ 1-v for v in mean_power]

pd.DataFrame({'mean':mean, 'mean_power':mean_power, 'activity':activity}).plot()

#### 7. Group별 frame개수, 시작시간, 재생시간 계산
- 사전준비 단계에서 구했던 tot_duration(재생시간), tot_frames(frame 개수) 사용하여 frame개수, 시작시간, 재생시간 항목 생성

In [ ]:
for group_number, group_df in df_takes:

    min_frame_number = group_df.index.min()+1
    max_frame_number = group_df.index.max()+1

    frame_count = max_frame_number - min_frame_number + 1
    start = tot_duration*(min_frame_number/tot_frames)
    duration = tot_duration*(frame_count/tot_frames)

    print(frame_count, start, duration)

#### 8. Group DataFrame 생성

In [ ]:
# 사전준비 단계에 구했던 tot_duration(재생시간), tot_frames(frame 개수) 사용

groups = []

for group_number, group_df in df_takes:

    min_frame_number = group_df.index.min()+1
    max_frame_number = group_df.index.max()+1
    min_similarity = group_df['similarity'].min()

    activity_intensity = 1 - group_df[group_df['similarity'] >\
                                min_similarity]['similarity'].mean()**100

    frame_count = max_frame_number - min_frame_number + 1
    start = tot_duration*(min_frame_number/tot_frames)
    duration = tot_duration*(frame_count/tot_frames)

    # dictionary를 append하는 경우 key값이 column명이 됨
    groups.append({
        'group_number': group_number,
        'group_name': f"Take #{group_number}",
        'min_frame_number': min_frame_number,
        'max_frame_number': max_frame_number,
        'frame_count' : frame_count,
        'start' : "{:.2f}".format(start),
        'duration' : "{:.2f}".format(duration),
        'min_similarity': min_similarity,
        'activity_intensity':activity_intensity
    })

# group 정보로 DataFrame 생성
group_df = pd.DataFrame(groups)

print(group_df)

#### 9. DataFrame을 csv파일에 저장

In [ ]:
csv = group_df.to_csv(path_or_buf = "video_grouping.csv", 
                      columns=[
                          'group_name', 
                          'frame_count', 
                          'start', 
                          'duration', 
                          'min_frame_number', 
                          'max_frame_number', 
                          'activity_intensity',
                          'min_similarity', 
                          ], 
                      index=False)